#### 1. 필요한 라이브러리 설치
- 먼저, 필요한 라이브러리를 설치합니다. 이 셀을 실행하여 transformers, torch, sentencepiece 등을 설치하세요.
- !pip install transformers torch sentencepiece

In [11]:
import torch
from transformers import AutoTokenizer, BartForConditionalGeneration, AdamW
from torch.utils.data import TensorDataset, DataLoader

# 장치 설정 (GPU 사용 가능 시 GPU, 아니면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 장치: {device}")


사용 중인 장치: cuda


In [12]:
train_data = [
    {"input": "배가 고픈데 뭐 먹을까?", "output": "한식은 어때요? 따뜻한 국밥이 좋을 것 같아요."},
    {"input": "점심 추천해줘", "output": "가벼운 샐러드나 샌드위치는 어떠세요?"},
    {"input": "저녁 뭐 먹을지 고민돼", "output": "맛있는 파스타나 스테이크는 어떠신가요?"},
    {"input": "달콤한 디저트 먹고 싶어", "output": "초콜릿 케이크나 아이스크림을 추천드려요."},
    {"input": "시원한 거 마시고 싶어", "output": "아이스 아메리카노나 레몬 에이드를 드셔보세요."},
    {"input": "매운 음식이 땡겨", "output": "매운 떡볶이나 불닭볶음면은 어떠세요?"},
    {"input": "간단하게 먹을 거 없어?", "output": "김밥이나 샌드위치를 드셔보세요."},
    {"input": "한국 전통 음식 추천해줘", "output": "비빔밥이나 불고기를 추천합니다."},
    {"input": "중국 음식이 먹고 싶어", "output": "짜장면이나 탕수육은 어떠세요?"},
    {"input": "일식 좋아해", "output": "스시나 우동을 드셔보세요."},
    {"input": "양식 먹고 싶어", "output": "스테이크나 피자를 추천합니다."},
    {"input": "채식주의자야", "output": "채소 샐러드나 두부 요리를 드셔보세요."},
    {"input": "디저트 카페 추천해줘", "output": "근처에 있는 베이커리 카페는 어떠세요?"},
    {"input": "건강식이 필요해", "output": "현미밥과 야채로 구성된 식단을 추천합니다."},
    {"input": "해산물 요리 먹고 싶어", "output": "회나 해물파전은 어떠세요?"},
    {"input": "친구들이랑 먹을 거 추천해줘", "output": "치킨이나 피자를 시켜 드세요."},
    {"input": "새로운 맛집 없어?", "output": "요즘 핫한 수제 버거 가게를 추천합니다."},
    {"input": "간식 추천해줘", "output": "과일이나 요거트를 추천드립니다."},
    {"input": "더운 날씨에 뭐 먹지?", "output": "시원한 냉면이나 팥빙수를 드셔보세요."},
    {"input": "감기 걸렸어", "output": "따뜻한 죽이나 수프를 드시는 게 좋겠어요."}
]


In [13]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    "gogamza/kobart-base-v2",
    bos_token='</s>',
    eos_token='</s>',
    unk_token='<unk>',
    pad_token='<pad>'
)

# pad_token이 eos_token과 동일한지 확인하고 다르게 설정
if tokenizer.pad_token == tokenizer.eos_token:
    tokenizer.add_special_tokens({'pad_token': '<pad>'})

inputs = []
labels = []

for pair in train_data:
    input_text = f"질문: {pair['input']}"
    output_text = f"답변: {pair['output']}"
    inputs.append(input_text)
    labels.append(output_text)

# 토크나이즈 및 텐서 변환 (max_length를 명시적으로 설정)
encodings = tokenizer(
    inputs,
    padding=True,
    truncation=True,
    max_length=128,  # 필요에 따라 조정 가능
    return_tensors='pt'
)

labels_encodings = tokenizer(
    labels,
    padding=True,
    truncation=True,
    max_length=128,  # 필요에 따라 조정 가능
    return_tensors='pt'
)

input_ids = encodings.input_ids
attention_mask = encodings.attention_mask
labels_ids = labels_encodings.input_ids

# 레이블에서 pad_token을 -100으로 마스킹하여 손실 계산에서 제외
labels_ids[labels_ids == tokenizer.pad_token_id] = -100


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
c:\ProgramData\anaconda3\envs\gnn_snn_torch\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
dataset = TensorDataset(input_ids, attention_mask, labels_ids)
loader = DataLoader(dataset, batch_size=4, shuffle=True)


In [15]:
model = BartForConditionalGeneration.from_pretrained("gogamza/kobart-base-v2")
model.resize_token_embeddings(len(tokenizer))  # pad_token 추가 후 토큰 임베딩 크기 조정
model.to(device)
model.train()

optimizer = AdamW(model.parameters(), lr=5e-5)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
c:\ProgramData\anaconda3\envs\gnn_snn_torch\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
epochs = 100  # 에포크 수 조정 가능

for epoch in range(epochs):
    print(f"에포크 {epoch+1}/{epochs} 진행 중...")
    total_loss = 0
    for batch in loader:
        optimizer.zero_grad()
        input_ids_batch = batch[0].to(device)
        attention_mask_batch = batch[1].to(device)
        labels_batch = batch[2].to(device)

        outputs = model(
            input_ids=input_ids_batch,
            attention_mask=attention_mask_batch,
            labels=labels_batch
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    print(f"에포크 {epoch+1} 완료, 평균 손실 값: {avg_loss:.4f}")


에포크 1/100 진행 중...
에포크 1 완료, 평균 손실 값: 9.3809
에포크 2/100 진행 중...
에포크 2 완료, 평균 손실 값: 4.3060
에포크 3/100 진행 중...
에포크 3 완료, 평균 손실 값: 2.3661
에포크 4/100 진행 중...
에포크 4 완료, 평균 손실 값: 1.6272
에포크 5/100 진행 중...
에포크 5 완료, 평균 손실 값: 1.1460
에포크 6/100 진행 중...
에포크 6 완료, 평균 손실 값: 0.8395
에포크 7/100 진행 중...
에포크 7 완료, 평균 손실 값: 0.6116
에포크 8/100 진행 중...
에포크 8 완료, 평균 손실 값: 0.4987
에포크 9/100 진행 중...
에포크 9 완료, 평균 손실 값: 0.3954
에포크 10/100 진행 중...
에포크 10 완료, 평균 손실 값: 0.3236
에포크 11/100 진행 중...
에포크 11 완료, 평균 손실 값: 0.2691
에포크 12/100 진행 중...
에포크 12 완료, 평균 손실 값: 0.2364
에포크 13/100 진행 중...
에포크 13 완료, 평균 손실 값: 0.1419
에포크 14/100 진행 중...
에포크 14 완료, 평균 손실 값: 0.1746
에포크 15/100 진행 중...
에포크 15 완료, 평균 손실 값: 0.1264
에포크 16/100 진행 중...
에포크 16 완료, 평균 손실 값: 0.0846
에포크 17/100 진행 중...
에포크 17 완료, 평균 손실 값: 0.0692
에포크 18/100 진행 중...
에포크 18 완료, 평균 손실 값: 0.0622
에포크 19/100 진행 중...
에포크 19 완료, 평균 손실 값: 0.0366
에포크 20/100 진행 중...
에포크 20 완료, 평균 손실 값: 0.0396
에포크 21/100 진행 중...
에포크 21 완료, 평균 손실 값: 0.0235
에포크 22/100 진행 중...
에포크 22 완료, 평균 손실 값: 0.0359
에포크 23

In [17]:
model.save_pretrained("./food_chatbot_model")
tokenizer.save_pretrained("./food_chatbot_model")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}


('./food_chatbot_model\\tokenizer_config.json',
 './food_chatbot_model\\special_tokens_map.json',
 './food_chatbot_model\\tokenizer.json')

In [18]:
from transformers import BartForConditionalGeneration, AutoTokenizer

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("./food_chatbot_model")
model = BartForConditionalGeneration.from_pretrained("./food_chatbot_model")
model.to(device)
model.eval()


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [21]:
def generate_response(input_text, mode='few-shot'):
    if mode == 'zero-shot':
        prompt = f"질문: {input_text}\n답변: "
    elif mode == 'few-shot':
        # Few-Shot을 위해 몇 가지 예시를 추가
        few_shot_examples = (
            "질문: 배가 고픈데 뭐 먹을까?\n답변: 한식은 어때요? 따뜻한 국밥이 좋을 것 같아요.\n"
            "질문: 점심 추천해줘\n답변: 가벼운 샐러드나 샌드위치는 어떠세요?\n"
            "질문: 저녁 뭐 먹을지 고민돼\n답변: 맛있는 파스타나 스테이크는 어떠신가요?\n"
        )
        prompt = f"{few_shot_examples}질문: {input_text}\n답변: "
    else:
        prompt = f"질문: {input_text}\n답변: "

    input_ids = tokenizer.encode(prompt, return_tensors='pt', truncation=True, max_length=128).to(device)
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=128,  # 적절한 max_length 설정
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=50,
            top_p=0.92,
            temperature=0.6,
            repetition_penalty=1.2,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True
        )
    output = tokenizer.decode(
        output_ids[0],
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True  # 경고를 제거하기 위해 추가
    )
    
    # 디버깅을 위한 출력
    # print(f"DEBUG: Generated Output: {output}")

    # 답변 부분만 추출 (마지막 '답변:' 이후 텍스트)
    if "답변:" in output:
        response = output.rsplit("답변:", 1)[-1].strip()
    else:
        response = "죄송합니다. 이해하지 못했어요."

    return response


In [22]:
print("음식 추천 챗봇입니다. 종료하려면 '종료'를 입력하세요.")

while True:
    user_input = input("사용자: ")
    if user_input.strip().lower() == "종료":
        print("챗봇: 대화를 종료합니다. 좋은 하루 되세요!")
        break
    # mode를 'zero-shot' 또는 'few-shot'으로 설정
    response = generate_response(user_input, mode='zero-shot')
    print(f"챗봇: {response}")


음식 추천 챗봇입니다. 종료하려면 '종료'를 입력하세요.
챗봇: 가벼운 샐러드나 샌드위치는 어떠세요? 따뜻한 국밥이 좋을 것 같아요. 따뜻한 국밥이나 샌드위치는 어떠세요? 따뜻한 국밥이 좋겠어요.
챗봇: 시원한 냉면이나 팥빙수를 드셔보세요.
챗봇: 시원한 냉면이나 팥빙수를 드셔보세요.
챗봇: 대화를 종료합니다. 좋은 하루 되세요!
